# MCCoder MCEval Dataset for evaluation


## Preparation


### Generate JSON dataset file from excel, python source code and log file.


#### Generate Json file from python source file


In [2]:
import pandas as pd
import os
import json

# Load the Excel file
excel_file_path = 'docs/WMX3API_MCEval_Evaluation_Dataset.xlsx'         # evaluation dataset
df = pd.read_excel(excel_file_path)

# Initialize an empty list to store the data for JSON
data_list = []

# Iterate through each row in the dataframe
for index, row in df.iterrows():
    task_id = row['TaskId']
    # Construct the path to the Python file
    py_file_path = f'MCEval_Files/{task_id}_MCEval.py'
    
    # Read the Python file to get the CanonicalCode
    if os.path.exists(py_file_path):
        with open(py_file_path, 'r') as file:
            canonical_code = file.read()
    else:
        canonical_code = ''
    
    # Create a dictionary for the current row
    row_data = row.to_dict()
    row_data['CanonicalCode'] = canonical_code
    
    # Append the dictionary to the list
    data_list.append(row_data)

# Construct the output JSON file path
json_file_path = excel_file_path.replace('.xlsx', '.json')

# Write the data to a JSON file
with open(json_file_path, 'w') as json_file:
    json.dump(data_list, json_file, indent=4)

print(f"JSON file has been generated at {json_file_path}")


JSON file has been generated at docs/WMX3API_MCEval_Evaluation_Dataset.json


/opt/anaconda3/envs/mc/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Sparkline Group extension is not supported and will be removed
  warn(msg)


#### Generate Json file from python function source excel file

In [3]:
import pandas as pd
import os
import json

# Load the Excel file
excel_file_path = 'docs/WMX3API_FunctionPython.xlsx'         # Python function list for self-correction
df = pd.read_excel(excel_file_path)

# Initialize an empty list to store the data for JSON
data_list = []

# Iterate through each row in the dataframe
for index, row in df.iterrows():

    # Create a dictionary for the current row
    row_data = row.to_dict()
    
    # Append the dictionary to the list
    data_list.append(row_data)

# Construct the output JSON file path
json_file_path = excel_file_path.replace('.xlsx', '.json')

# Write the data to a JSON file
with open(json_file_path, 'w') as json_file:
    json.dump(data_list, json_file, indent=4)

print(f"Python Function list JSON file has been generated at {json_file_path}")


Python Function list JSON file has been generated at docs/WMX3API_FunctionPython.json


#### Generate Json file from python source file with motion log


In [6]:
import pandas as pd
import os
import json
import socket
from time import *
from tqdm import tqdm

# Load the Excel file
excel_file_path = 'docs/WMX3API_MCEval_Dataset.xlsx'
df = pd.read_excel(excel_file_path)

# Initialize an empty list to store the data for JSON
data_list = []

# Function to modify the Python code with logging
def add_log_code(py_code, task_id, axis):
    lines = py_code.split('\n')
    modified_code = []
    i = 0
    # Add logging start code
    while i < len(lines):
        modified_code.append(lines[i])
        if '.StartHome' in lines[i]:
            i += 1
            while i < len(lines) and lines[i].strip() != '':
                modified_code.append(lines[i])
                i += 1
            if i < len(lines):
                modified_code.append(f'''
                                     
    # <log ---------------------------------------------------------------------------                                                                 
    WMX3Log = Log(Wmx3Lib)

    axislist = {axis}                           
    num = len(axislist)

    # Set Axis numbers and control variables of log
    cmLogIn_0 = CoreMotionLogInput()
    cmLogIn_0.axisSelection.axisCount = num
    for i in range(0, num):
        cmLogIn_0.axisSelection.SetAxis(i, axislist[i])

    # Control variables to log
    cmLogIn_0.axisOptions.commandPos = 1
    cmLogIn_0.axisOptions.feedbackPos = 0
    cmLogIn_0.axisOptions.commandVelocity = 0
    cmLogIn_0.axisOptions.feedbackVelocity = 0

    # Set up log time
    option = LogChannelOptions()
    option.samplingPeriodInCycles = 1
    option.samplingTimeMilliseconds = 1000000  

    ret=WMX3Log.SetLogOption(0, option)
    if ret!=0:
        print('SetLogOption error code is ' + str(ret) + ': ' + WMX3Log.ErrorToString(ret))
        return
    sleep(0.1)
    ret = WMX3Log.SetCustomLog(0,cmLogIn_0)
    if ret!=0:
        print('SetCustomLog error code is ' + str(ret) + ': ' + WMX3Log.ErrorToString(ret))
        return
    sleep(0.1)

    # Set log file address
    path_0 = LogFilePath()
    WMX3Log.GetLogFilePath(0)
    path_0.dirPath = "C:\\\\"
    path_0.fileName = f"{task_id}_MCEval_Log.txt"
    ret = WMX3Log.SetLogFilePath(0, path_0)
    if ret!=0:
        print('SetLogFilePath error code is ' + str(ret) + ': ' + WMX3Log.ErrorToString(ret))
        return

    # Stop log just in case logging is on.
    ret = WMX3Log.StopLog(0)
    sleep(0.01)

    # Start log
    ret = WMX3Log.StartLog(0)
    if ret!=0:
        print('StartLog error code is ' + str(ret) + ': ' + WMX3Log.ErrorToString(ret))
        return
    sleep(0.01)
    # log> ---------------------------------------------------------------------------   
    
                ''')
        i += 1

    # Add logging stop code
    i = len(lines) - 1
    while i >= 0:
        if '.SetServoOn' in lines[i]:
            while i >= 0 and lines[i].strip() != '':
                i -= 1
            if i >= 0:
                modified_code.insert(i + 1, '''
                                     
    # <log --------------------------------------------------------------------------- 
    sleep(0.1)                                                                    
    # Stop log
    ret = WMX3Log.StopLog(0)
    if ret!=0:
        print('StopLog error code is ' + str(ret) + ': ' + WMX3Log.ErrorToString(ret))
        return

    for axisNo in axislist:                                
        ret = Wmx3Lib_cm.home.StartHome(axisNo)
        if ret!=0:
            print('StartHome error code is ' + str(ret) + ': ' + Wmx3Lib_cm.ErrorToString(ret))
            return
        Wmx3Lib_cm.motion.Wait(axisNo)                                                                  
    # log> ---------------------------------------------------------------------------     
                                     
                ''')
            break
        i -= 1

    return '\n'.join(modified_code)

# Function to send code and log data to socket server
def SendCodeLog(code_data_log):
    # 1. Create socket
    tcp_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    # 2. Connect server
    server_addr = ("10.211.55.5", 8888)
    tcp_socket.connect(server_addr)
    print("Socket connected")

    import time

    # 3. Send data
    tcp_socket.send(code_data_log.encode("utf-8"))

    time.sleep(0.1)

    send_data = 'exit'
    tcp_socket.send(send_data.encode("utf-8"))

    # 4. Receive data from server
    tcp_remsg = tcp_socket.recv(1024)
    print(tcp_remsg.decode("utf-8"))

    # 5. Close socket
    tcp_socket.close()
    print("Socket end")

# Iterate through each row in the dataframe with a progress bar
for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing rows"):
    task_id = row['TaskId']
    # Construct the path to the Python file
    py_file_path = f'MCEval_Files/{task_id}_MCEval.py'
    
    # Read the Python file to get the CanonicalCode
    if os.path.exists(py_file_path):
        with open(py_file_path, 'r') as file:
            py_code = file.read()
    else:
        py_code = ''
    
    #If it is about motion log
    print('endp: ' + str(row['EndPoint']))
    if (str(row['EndPoint']) != 'nan'):
        # Extract axis from the first row of 'EndPoint'
        try:
            axis = list(range(len(eval(row['EndPoint'])[0])))
            axis = eval(row['EndPoint'])[0]
            print("axisl: " + str(axis))
            if not axis:
                continue
        except Exception as e:
            print(f"Error processing axis for task {task_id}: {e}")
            continue

        # Add log code to the Python file content
        modified_code = add_log_code(py_code, task_id, axis)
    else:
        modified_code = py_code
    
    # Save the modified code to a new file
    modified_file_path = f'MCEval_Files/{task_id}_MCEval_WithLog.py'
    with open(modified_file_path, 'w') as file:
        file.write(modified_code)
    
    # Send the modified code to the socket server for each task
    SendCodeLog(modified_code)
    sleep(0.3)

    # Create a dictionary for the current row
    row_data = row.to_dict()
    row_data['CanonicalCode'] = py_code
    
    # Append the dictionary to the list
    data_list.append(row_data)

    sleep(0.2)

# Construct the output JSON file path
json_file_path = excel_file_path.replace('.xlsx', '.json')

# Write the data to a JSON file
with open(json_file_path, 'w') as json_file:
    json.dump(data_list, json_file, indent=4)

print(f"JSON file has been generated at {json_file_path}")


FileNotFoundError: [Errno 2] No such file or directory: 'docs/WMX3API_MCEval_Dataset.xlsx'

#### Generate Json file from python source file with motion log, adding error stastics


In [1]:
import pandas as pd
import os
import json
import socket
from time import sleep
from tqdm import tqdm

# Load the Excel file
excel_file_path = 'docs/WMX3API_MCEval_Dataset.xlsx'
df = pd.read_excel(excel_file_path)

# Initialize an empty list to store the data for JSON
data_list = []

# Lists to store TaskIds with different errors
error_task_ids = []
syntax_error_task_ids = []

# Function to modify the Python code with logging
def add_log_code(py_code, task_id, axis):
    lines = py_code.split('\n')
    modified_code = []
    i = 0
    # Add logging start code
    while i < len(lines):
        modified_code.append(lines[i])
        if '.StartHome' in lines[i]:
            i += 1
            while i < len(lines) and lines[i].strip() != '':
                modified_code.append(lines[i])
                i += 1
            if i < len(lines):
                modified_code.append(f'''
    # <log ---------------------------------------------------------------------------                                                                 
    WMX3Log = Log(Wmx3Lib)

                                     
    # Stop log just in case logging is on.
    ret = WMX3Log.StopLog(0)
    sleep(0.01)
                                     
    axislist = {axis}                           
    num = len(axislist)

    # Set Axis numbers and control variables of log
    cmLogIn_0 = CoreMotionLogInput()
    cmLogIn_0.axisSelection.axisCount = num
    for i in range(0, num):
        cmLogIn_0.axisSelection.SetAxis(i, axislist[i])

    # Control variables to log
    cmLogIn_0.axisOptions.commandPos = 1
    cmLogIn_0.axisOptions.feedbackPos = 0
    cmLogIn_0.axisOptions.commandVelocity = 0
    cmLogIn_0.axisOptions.feedbackVelocity = 0

    # Set up log time
    option = LogChannelOptions()
    option.samplingPeriodInCycles = 1
    option.samplingTimeMilliseconds = 1000000
    option.precision = 3

    ret=WMX3Log.SetLogOption(0, option)
    if ret!=0:
        print('SetLogOption error code is ' + str(ret) + ': ' + WMX3Log.ErrorToString(ret))
        return
    sleep(0.1)
    ret = WMX3Log.SetCustomLog(0,cmLogIn_0)
    if ret!=0:
        print('SetCustomLog error code is ' + str(ret) + ': ' + WMX3Log.ErrorToString(ret))
        return
    sleep(0.1)

    # Set log file address
    path_0 = LogFilePath()
    WMX3Log.GetLogFilePath(0)
    path_0.dirPath = "C:\\\\"
    path_0.fileName = f"{task_id}_MCEval_Log.txt"
    ret = WMX3Log.SetLogFilePath(0, path_0)
    if ret!=0:
        print('SetLogFilePath error code is ' + str(ret) + ': ' + WMX3Log.ErrorToString(ret))
        return

    # Start log
    ret = WMX3Log.StartLog(0)
    if ret!=0:
        print('StartLog error code is ' + str(ret) + ': ' + WMX3Log.ErrorToString(ret))
        return
    sleep(0.01)
    # log> ---------------------------------------------------------------------------   
    
                ''')
        i += 1

    # Add logging stop code
    i = len(lines) - 1
    while i >= 0:
        if '.SetServoOn' in lines[i]:
            while i >= 0 and lines[i].strip() != '':
                i -= 1
            if i >= 0:
                modified_code.insert(i + 1, '''
    # <log --------------------------------------------------------------------------- 
    sleep(0.1)                                                                    
    # Stop log
    ret = WMX3Log.StopLog(0)
    if ret!=0:
        print('StopLog error code is ' + str(ret) + ': ' + WMX3Log.ErrorToString(ret))
        return

    for axisNo in axislist:                                
        ret = Wmx3Lib_cm.home.StartHome(axisNo)
        if ret!=0:
            print('StartHome error code is ' + str(ret) + ': ' + Wmx3Lib_cm.ErrorToString(ret))
            return
        Wmx3Lib_cm.motion.Wait(axisNo)                                                                  
    # log> ---------------------------------------------------------------------------     
                                     
                ''')
            break
        i -= 1

    return '\n'.join(modified_code)

# Function to send code and log data to socket server
def SendCodeLog(code_data_log, task_id):
    # 1. Create socket
    tcp_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    # 2. Connect server
    server_addr = ("10.211.55.5", 8888)
    tcp_socket.connect(server_addr)
    print("Socket connected")

    # 3. Send data
    tcp_socket.send(code_data_log.encode("utf-8"))

    sleep(0.1)

    send_data = 'exit'
    tcp_socket.send(send_data.encode("utf-8"))

    # 4. Receive data from server
    tcp_remsg = tcp_socket.recv(1024).decode("utf-8")
    print(tcp_remsg)

    # Check for errors in the response
    if "error" in tcp_remsg:
        if "syntax error" in tcp_remsg:
            syntax_error_task_ids.append(task_id)
        else:
            error_task_ids.append(task_id)

    # 5. Close socket
    tcp_socket.close()
    print("Socket end")

# Iterate through each row in the dataframe with a progress bar
for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing rows"):
    task_id = row['TaskId']
    # if(task_id != 32):
    #     continue
    
    # Construct the path to the Python file
    py_file_path = f'MCEval_Files/{task_id}_MCEval.py'
    
    # Read the Python file to get the CanonicalCode
    if os.path.exists(py_file_path):
        with open(py_file_path, 'r') as file:
            py_code = file.read()
    else:
        py_code = ''
    
    # If it is about motion log
    print('endp: ' + str(row['EndPoint']))
    if str(row['EndPoint']) != 'nan':
        # Extract axis from the first row of 'EndPoint'
        try:
            axis = list(range(len(eval(row['EndPoint'])[0])))
            axis = eval(row['EndPoint'])[0]
            print("axisl: " + str(axis))
            if not axis:
                continue
        except Exception as e:
            print(f"Error processing axis for task {task_id}: {e}")
            continue

        # Add log code to the Python file content
        modified_code = add_log_code(py_code, task_id, axis)
    else:
        modified_code = py_code
    
    # Save the modified code to a new file
    modified_file_path = f'MCEval_Files/{task_id}_MCEval_WithLog.py'
    with open(modified_file_path, 'w') as file:
        file.write(modified_code)
    
    # Send the modified code to the socket server for each task
    SendCodeLog(modified_code, task_id)
    sleep(0.3)

    # Create a dictionary for the current row
    row_data = row.to_dict()
    row_data['CanonicalCode'] = py_code
    
    # Append the dictionary to the list
    data_list.append(row_data)

    sleep(0.2)

# Construct the output JSON file path
json_file_path = excel_file_path.replace('.xlsx', '.json')

# Write the data to a JSON file
with open(json_file_path, 'w') as json_file:
    json.dump(data_list, json_file, indent=4)

# Calculate accuracy
total_tasks = len(df)
error_count = len(error_task_ids)
syntax_error_count = len(syntax_error_task_ids)
success_count = total_tasks - (error_count + syntax_error_count)
accuracy = (success_count / total_tasks) * 100

print(f"JSON file has been generated at {json_file_path}")
print(f"Total tasks: {total_tasks}")
print(f"Error tasks: {error_count}")
print(f"Syntax error tasks: {syntax_error_count}")
print(f"Successful tasks: {success_count}")
print(f"Accuracy: {accuracy:.2f}%")
print(f"Tasks with errors: {error_task_ids}")
print(f"Tasks with syntax errors: {syntax_error_task_ids}")


/opt/anaconda3/lib/python3.11/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Sparkline Group extension is not supported and will be removed
  for idx, row in parser.parse():
Processing rows:   0%|          | 0/69 [00:00<?, ?it/s]

endp: [[0],
[180]]
axisl: [0]
Socket connected
149
codedeta:Program begin.
Program End.
, Elapsed_time: 4.464
codeerr:
Socket end


Processing rows:   1%|▏         | 1/69 [00:05<05:45,  5.08s/it]

endp: [[1],
[200]]
axisl: [1]
Socket connected
150
codedeta:Program begin.
Program End.
, Elapsed_time: 3.523
codeerr:
Socket end


Processing rows:   3%|▎         | 2/69 [00:09<05:03,  4.52s/it]

...

endp: [[0,1,2],
[0,0,0]]
axisl: [0, 1, 2]
Socket connected
216
codedeta:Program begin.
Program End.
, Elapsed_time: 9.258
codeerr:
Socket end


Processing rows:  99%|█████████▊| 68/69 [07:11<00:06,  6.70s/it]

endp: [[0,1,2,3],
[0,0,0,100]]
axisl: [0, 1, 2, 3]
Socket connected
217
codedeta:Program begin.
Program End.
, Elapsed_time: 5.961
codeerr:
Socket end


Processing rows: 100%|██████████| 69/69 [07:18<00:00,  6.35s/it]

JSON file has been generated at docs/WMX3API_MCEval_Dataset.json
Total tasks: 69
Error tasks: 0
Syntax error tasks: 0
Successful tasks: 69
Accuracy: 100.00%
Tasks with errors: []
Tasks with syntax errors: []


### MCEval: code snippet to runnable code. (!!)

In [1]:
def process_file(input_filepath):
    """
    This function takes the path to a Python file, extracts the line starting with '# Axes =', and inserts it into stringA.
    Then it concatenates stringA, the content of the input file, and stringC, and writes the result to a new file.
    """
    stringA = '''
# Define Axes


#WMX3 python library
from WMX3ApiPython import *
from time import *

INFINITE = int(0xFFFFFFFF)

def main():
    Wmx3Lib = WMX3Api()
    CmStatus = CoreMotionStatus()
    Wmx3Lib_cm = CoreMotion(Wmx3Lib)
    print('Program begin.')
    sleep(0.1)

    # Create devices. 
    ret = Wmx3Lib.CreateDevice('C:\\\\Program Files\\\\SoftServo\\\\WMX3', DeviceType.DeviceTypeNormal, INFINITE)
    if ret!=0:
        print('CreateDevice error code is ' + str(ret) + ': ' + Wmx3Lib.ErrorToString(ret))
        return

    # Set Device Name.
    Wmx3Lib.SetDeviceName('WMX3initTest')

    # Start Communication.
    ret = Wmx3Lib.StartCommunication(INFINITE)
    if ret!=0:
        print('StartCommunication error code is ' + str(ret) + ': ' + Wmx3Lib.ErrorToString(ret))
        return

    # Import and set all the preset motion parameters.
    ret=Wmx3Lib_cm.config.ImportAndSetAll("C:\\\\Program Files\\\\SoftServo\\\\WMX3\\\\wmx_parameters.xml")
    if ret != 0:
        print('ImportAndSetAll Parameters error code is ' + str(ret) + ': ' + Wmx3Lib_cm.ErrorToString(ret))
    sleep(0.5)
 
    # Clear alarms, set servos on, and perform homing for Axes
    for axis in Axes:
        # Clear the amplifier alarm
        timeoutCounter = 0
        while True:
            # GetStatus -> First return value : Error code, Second return value: CoreMotionStatus
            ret, CmStatus = Wmx3Lib_cm.GetStatus()
            if not CmStatus.GetAxesStatus(axis).ampAlarm:
                break
            ret = Wmx3Lib_cm.axisControl.ClearAmpAlarm(axis)
            sleep(0.5)
            timeoutCounter += 1
            if timeoutCounter > 5:
                break
        if timeoutCounter > 5:
            print(f'Clear axis {axis} alarm fails!')
            return

        # Set servo on for Axis
        ret = Wmx3Lib_cm.axisControl.SetServoOn(axis, 1)
        timeoutCounter = 0
        while True:
            # GetStatus -> First return value : Error code, Second return value: CoreMotionStatus
            ret, CmStatus = Wmx3Lib_cm.GetStatus()
            if (CmStatus.GetAxesStatus(axis).servoOn):
                break
            sleep(0.4)
            timeoutCounter += 1
            if (timeoutCounter > 5):
                break
        if (timeoutCounter > 5):
            print('Set servo on for axis {axis} fails!')
            return

        # Sleep is a must between SetServoOn and Homing
        sleep(0.1)

        # Homing
        homeParam = Config_HomeParam()
        ret, homeParam = Wmx3Lib_cm.config.GetHomeParam(axis)
        homeParam.homeType = Config_HomeType.CurrentPos

        # SetHomeParam -> First return value: Error code, Second return value: param error
        ret, homeParamError = Wmx3Lib_cm.config.SetHomeParam(axis, homeParam)

        ret = Wmx3Lib_cm.home.StartHome(axis)
        if ret != 0:
            print(f'StartHome error code for axis {axis} is ' + str(ret) + ': ' + Wmx3Lib_cm.ErrorToString(ret))
            return
        Wmx3Lib_cm.motion.Wait(axis)

    # <log ---------------------------------------------------------------------------                                                                 
    WMX3Log = Log(Wmx3Lib)
                                 
    # Stop log just in case logging is on.
    ret = WMX3Log.StopLog(0)
    sleep(0.01)
                                     
    axislist = Axes                           
    num = len(axislist)

    # Set Axis numbers and control variables of log
    cmLogIn_0 = CoreMotionLogInput()
    cmLogIn_0.axisSelection.axisCount = num
    for i in range(0, num):
        cmLogIn_0.axisSelection.SetAxis(i, axislist[i])

    # Control variables to log
    cmLogIn_0.axisOptions.commandPos = 1
    cmLogIn_0.axisOptions.feedbackPos = 0
    cmLogIn_0.axisOptions.commandVelocity = 0
    cmLogIn_0.axisOptions.feedbackVelocity = 0

    # Set up log time
    option = LogChannelOptions()
    option.samplingPeriodInCycles = 1
    option.samplingTimeMilliseconds = 1000000
    option.precision = 3

    ret=WMX3Log.SetLogOption(0, option)
    if ret!=0:
        print('SetLogOption error code is ' + str(ret) + ': ' + WMX3Log.ErrorToString(ret))
        return
    sleep(0.1)
    ret = WMX3Log.SetCustomLog(0,cmLogIn_0)
    if ret!=0:
        print('SetCustomLog error code is ' + str(ret) + ': ' + WMX3Log.ErrorToString(ret))
        return
    sleep(0.1)

    # Set log file address
    path_0 = LogFilePath()
    WMX3Log.GetLogFilePath(0)
    path_0.dirPath = "C:\\\\"
    path_0.fileName = f"filename_Log.txt"
    ret = WMX3Log.SetLogFilePath(0, path_0)
    if ret!=0:
        print('SetLogFilePath error code is ' + str(ret) + ': ' + WMX3Log.ErrorToString(ret))
        return

    # Start log
    ret = WMX3Log.StartLog(0)
    if ret!=0:
        print('StartLog error code is ' + str(ret) + ': ' + WMX3Log.ErrorToString(ret))
        return
    sleep(0.01)
    # log> ---------------------------------------------------------------------------   


'''

    stringC = '''
# <log --------------------------------------------------------------------------- 
    sleep(0.1)                                                                    
    # Stop log
    ret = WMX3Log.StopLog(0)
    if ret!=0:
        print('StopLog error code is ' + str(ret) + ': ' + WMX3Log.ErrorToString(ret))
        return

    for axisNo in axislist:                                
        ret = Wmx3Lib_cm.home.StartHome(axisNo)
        if ret!=0:
            print('StartHome error code is ' + str(ret) + ': ' + Wmx3Lib_cm.ErrorToString(ret))
            return
        Wmx3Lib_cm.motion.Wait(axisNo)                                                                  
    # log> ---------------------------------------------------------------------------     
                                     
                
    # Set servo off for Axes
    for axis in Axes:
        ret = Wmx3Lib_cm.axisControl.SetServoOn(axis, 0)
        if ret != 0:
            print(f'SetServoOn to off error code for axis {axis} is ' + str(ret) + ': ' + Wmx3Lib_cm.ErrorToString(ret))
            return

    # Stop Communication.
    ret = Wmx3Lib.StopCommunication(INFINITE)
    if ret!=0:
        print('StopCommunication error code is ' + str(ret) + ': ' + Wmx3Lib.ErrorToString(ret))
        return

    # Close Device.
    ret = Wmx3Lib.CloseDevice()
    if ret!=0:
        print('CloseDevice error code is ' + str(ret) + ': ' + Wmx3Lib.ErrorToString(ret))
        return

    print('Program End.')

if __name__ == '__main__':
    main()

'''

    with open(input_filepath, 'r') as file:
        lines = file.readlines()

    axes_line = None
    for line in lines:
        if line.strip().startswith('# Axes ='):
            axes_line = line.strip().lstrip('# ').strip()
            break

    if axes_line:
        stringA = stringA.replace('# Define Axes', f'# Define Axes\n{axes_line}')

    stringB = ''.join(lines)
    filename = input_filepath.split('/')[-1].split('.')[0]
    stringA = stringA.replace("filename_Log.txt", f"{filename}_SampleCode_Log.txt")
    
    output_content = stringA + stringB + stringC
    output_filepath = input_filepath.replace('.py', '_Runnable.py')
    
    with open(output_filepath, 'w') as output_file:
        output_file.write(output_content)

    print(f'Processed file saved to {output_filepath}')


for i in range(1, 70):
    input_filepath = f'MCEval_Files/{i}_MCEval.py'
    process_file(input_filepath)


Processed file saved to MCEval_Files/1_MCEval_Runnable.py
Processed file saved to MCEval_Files/2_MCEval_Runnable.py
Processed file saved to MCEval_Files/3_MCEval_Runnable.py
Processed file saved to MCEval_Files/4_MCEval_Runnable.py
Processed file saved to MCEval_Files/5_MCEval_Runnable.py
Processed file saved to MCEval_Files/6_MCEval_Runnable.py
Processed file saved to MCEval_Files/7_MCEval_Runnable.py
Processed file saved to MCEval_Files/8_MCEval_Runnable.py
Processed file saved to MCEval_Files/9_MCEval_Runnable.py
Processed file saved to MCEval_Files/10_MCEval_Runnable.py
Processed file saved to MCEval_Files/11_MCEval_Runnable.py
Processed file saved to MCEval_Files/12_MCEval_Runnable.py
Processed file saved to MCEval_Files/13_MCEval_Runnable.py
Processed file saved to MCEval_Files/14_MCEval_Runnable.py
Processed file saved to MCEval_Files/15_MCEval_Runnable.py
Processed file saved to MCEval_Files/16_MCEval_Runnable.py
Processed file saved to MCEval_Files/17_MCEval_Runnable.py
Proces

 ### send runnable code and log data to socket server to run and check.

In [4]:
import socket
import os
from time import sleep
from tqdm import tqdm

# Lists to track error task IDs and types
syntax_error_task_ids = []
error_task_ids = []

# Function to send code and log data to socket server
def SendCodeLog(code_data_log, task_id):
    """
    Sends the given code and task ID to the server via a socket connection,
    and logs any errors encountered.
    """
    try:
        # 1. Create socket
        tcp_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

        # 2. Connect server
        server_addr = ("10.211.55.5", 8888)
        tcp_socket.connect(server_addr)
        print(f"Socket connected for task {task_id}")

        # 3. Send data
        tcp_socket.send(code_data_log.encode("utf-8"))

        sleep(0.1)

        send_data = 'exit'
        tcp_socket.send(send_data.encode("utf-8"))

        # 4. Receive data from server
        tcp_remsg = tcp_socket.recv(1024).decode("utf-8")
        print(f"Response for task {task_id}: {tcp_remsg}")

        # Check for errors in the response
        if "error" in tcp_remsg:
            if "syntax error" in tcp_remsg:
                syntax_error_task_ids.append(task_id)
            else:
                error_task_ids.append(task_id)

        # 5. Close socket
        tcp_socket.close()
        print(f"Socket end for task {task_id}")

    except Exception as e:
        print(f"Exception for task {task_id}: {e}")
        error_task_ids.append(task_id)

# Main script to read files, send code, and track errors
def main():
    base_path = 'MCEval_Files'
    total_tasks = 69

    # Loop through each task file
    for no in tqdm(range(1, total_tasks + 1), desc="Processing Tasks"):
        file_path = os.path.join(base_path, f'{no}_MCEval_Runnable.py')
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                code = file.read()
                SendCodeLog(code, no)
        except FileNotFoundError:
            print(f"File not found for task {no}")
            error_task_ids.append(no)
        except Exception as e:
            print(f"Exception while reading file for task {no}: {e}")
            error_task_ids.append(no)

    # Print summary of errors
    print(f"Total tasks with syntax errors: {len(syntax_error_task_ids)}")
    print(f"Tasks with syntax errors: {syntax_error_task_ids}")
    print(f"Total tasks with other errors: {len(error_task_ids)}")
    print(f"Tasks with other errors: {error_task_ids}")

if __name__ == "__main__":
    main()


Processing Tasks:   0%|          | 0/69 [00:00<?, ?it/s]

Socket connected for task 1


Processing Tasks:   1%|▏         | 1/69 [00:03<03:56,  3.48s/it]

Response for task 1: 258
codedeta:Program begin.
Program End.
, Elapsed_time: 3.378
codeerr:
Socket end for task 1
Socket connected for task 2


Processing Tasks:   3%|▎         | 2/69 [00:06<03:47,  3.39s/it]

...

Response for task 67: 324
codedeta:Program begin.
Program End.
, Elapsed_time: 4.500
codeerr:
Socket end for task 67
Socket connected for task 68


Processing Tasks:  99%|█████████▊| 68/69 [06:00<00:05,  5.52s/it]

Response for task 68: 325
codedeta:Program begin.
Program End.
, Elapsed_time: 8.182
codeerr:
Socket end for task 68
Socket connected for task 69


Processing Tasks: 100%|██████████| 69/69 [06:06<00:00,  5.31s/it]

Response for task 69: 326
codedeta:Program begin.
Program End.
, Elapsed_time: 5.624
codeerr:
Socket end for task 69
Total tasks with syntax errors: 0
Tasks with syntax errors: []
Total tasks with other errors: 0
Tasks with other errors: []
